In [1]:
import websocket, json
from binance.client import Client
from binance.enums import KLINE_INTERVAL_1MINUTE
from datetime import datetime
import keys

In [2]:
BINANCE_API = keys.BINANCE_API
BINANCE_SECRET = keys.BINANCE_SECRET

TEST_BINANCE_API = keys.TEST_BINANCE_API
TEST_BINANCE_SECRET = keys.TEST_BINANCE_SECRET

# for testnet trading
test_client = Client(TEST_BINANCE_API, TEST_BINANCE_SECRET)
test_client.API_URL = 'https://testnet.binance.vision/api'
# test_socket = "wss://testnet.binance.vision/ws/bnbusdt@kline_1m"

# for production and real prices
client = Client(BINANCE_API, BINANCE_SECRET)
socket = "wss://stream.binance.com:9443/ws/bnbusdt@kline_1m"

In [3]:
BASE = 'BNB'
QUOTE = 'USDT'
SYMBOL = BASE+QUOTE
LEVER = 0.1

In [4]:
def get_margin_isolated_account_quote_balance(symbol,client,quote):
    try:
        account = client.get_isolated_margin_account(symbols=symbol)
        if len(account['assets']) == 0:
            print("There is no isolated margin account {}. Need to create it".format(symbol))
            return -2
        else:
            if float(account['assets'][0]['quoteAsset']['borrowed']) == 0:
                quote_balance = float(account['assets'][0]['quoteAsset']['netAsset'])
                print("Quote net balance is {} {}".format(quote_balance,quote))
                return quote_balance 
            else:
                print("Quote borrowed > 0")
                return -3
    except Exception as e:
        print("get_margin_solated_account error: {}".format(e))
        return -1

In [5]:
def create_qoute_loan(symbol,client,quote,quote_net_balance,lever):
    try:
        amount = quote_net_balance*lever
        transaction = client.create_margin_loan(asset=quote,amount=amount,isIsolated='TRUE',symbol=symbol)
        if transaction['tranId'] > 0:
            account = client.get_isolated_margin_account(symbols=symbol)
            quote_loan = float(account['assets'][0]['quoteAsset']['borrowed'])
            quote_free = float(account['assets'][0]['quoteAsset']['free'])
            print("Loan {} is {}, free {} is {}".format(quote,quote_loan,quote,quote_free))
            return quote_free
    except Exception as e:
        print("get_qoute_loan error: {}".format(e))
        return -1

In [6]:
def repay_quote_loan(symbol,client,quote):
    try:
        account = client.get_isolated_margin_account(symbols=symbol)
        borrowed = float(account['assets'][0]['quoteAsset']['borrowed'])
        interest = float(account['assets'][0]['quoteAsset']['interest'])
        amount = borrowed+interest
        transaction = client.repay_margin_loan(asset=quote,amount=amount,isIsolated='TRUE',symbol=symbol)
        if transaction['tranId'] > 0:
            account = client.get_isolated_margin_account(symbols=symbol)
            borrowed = float(account['assets'][0]['quoteAsset']['borrowed'])
            net_balance = float(account['assets'][0]['quoteAsset']['netAsset'])
            if borrowed == 0:
                print("Repay is finished. Total net balance is {} {}".format(net_balance,QUOTE))
                return net_balance
            else:
                Print("Something wrong. Borrowed is {} {}".format(borrowed,QUOTE))
                return -2
    except Exception as e:
        print("repay_quote_loan error: {}".format(e))
        return -1

In [17]:
quote_net_balance = get_margin_isolated_account_quote_balance(SYMBOL,client,QUOTE)
quote_free_balance = create_qoute_loan(SYMBOL,client,QUOTE,quote_net_balance,LEVER)
new_quote_net_balance = repay_quote_loan(SYMBOL,client,QUOTE)

Quote net balance is 5.00889982 USDT
Loan USDT is 0.50088998, free USDT is 5.5097898
Repay is finished. Total net balance is 5.00886851 USDT


-3.1309999999784566e-05

In [ ]:
def margin_buy():
    # перевести все usdt со спотового аккаунта
    # проверить изолированный аккаунт и сделать, если его нет
    # занять денег с плечом
    # купить базовый актив

In [ ]:
def margin_sell()
    # продать базовый актив
    # отдать долг